In [1]:
import cv2
from matplotlib import pyplot as plt
%matplotlib inline
import pandas as pd
import math
import numpy as np
import os

In [2]:
AUDIO_RATE = 16000
y1, y2, y3, y4, x1, x2, x3, x4 = (128, 352, 128, 352, 68, 292, 428, 652)

In [3]:
import pims
video = pims.Video('Ses01F_impro01.avi')
frame = video[0:1]

In [8]:
def gen_video_array(vid):
    sess_id = vid[4]
    video_path = "E:/datasets/IEMOCAP_full_release.tar/IEMOCAP_full_release/IEMOCAP_full_release/Session{}/dialog/avi/DivX/{}.avi".format(sess_id,vid)
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    success=True
    frame_list = []
    while success:
        success, frame=cap.read()
        frame_list.append(frame)

    video_array = np.array(frame_list[:-1])
    del frame_list
    return video_array, fps

In [19]:
def save_arr2vid(arr, output_path, fps):
    print(arr[0][0])
    return
    out = cv2.VideoWriter(output_path, 
                          cv2.VideoWriter_fourcc(*'mp4v'), 
                          fps, (arr.shape[2], arr.shape[1]))
    for i in range(arr.shape[0]):
        data = arr[i, :, :, :]
        out.write(data)
    out.release()

In [6]:
def save_video_array(vid, output_dir, fps):
    sess_id = vid[4]
    video_path = "E:/datasets/IEMOCAP_full_release.tar/IEMOCAP_full_release/IEMOCAP_full_release/Session{}/dialog/avi/DivX/{}.avi".format(sess_id,vid)
    video = pims.Video(video_path)
    df = pd.read_excel('E:/datasets/preprocessed/extractionmap/cut_extractionmap{}.xlsx'.format(sess_id), sheet_name=vid)
    df["viframe"] = np.floor(df["iframe"]*fps/AUDIO_RATE).astype(int)
    df["vfframe"] = np.ceil(df["fframe"]*fps/AUDIO_RATE).astype(int)
    
    for index, row in df.iterrows():
        varr = np.asarray(video[row["viframe"]:row["vfframe"]])
        if row["speaker"] == "L":
            arr = varr[:, 
                       y1:y2, 
                       x1:x2, :]
        elif row["speaker"] == "R":
            arr = varr[:, 
                       y3:y4, 
                       x3:x4, :]
            
        img_stack_sm = []
        for idx in range(arr.shape[0]):
            img = arr[idx, :, :, :]
            img_sm = cv2.resize(img, (88, 88), interpolation=cv2.INTER_AREA)
            img_stack_sm.append(img_sm)
            
        img_stack_sm = np.array(img_stack_sm)
            
        # save video
        save_path = os.path.join(output_dir, "{}".format(row["smp_id"]))
        save_arr2vid(img_stack_sm, save_path+".mp4", fps)

In [9]:
varr, fps = gen_video_array("Ses01F_impro01")

In [11]:
varr.shape

(3367, 480, 720, 3)

In [8]:
# vr = save_video_array("Ses01F_impro01",
#                  "E:/datasets/preprocessed/face_raw",
#                  fps)

### Pipeline

In [15]:
import torch
import sys
sys.path.append("E:/university/FYT/repos/multi_modal_ser")
from utils.dataset import MMSERDataset
mmser_ds = torch.load("E:/datasets/preprocessed/dataset/mmser_ds.pt")
df_ = mmser_ds.df_

In [16]:
df_["fn"] = df_["smp_id"].apply(lambda x: x[:-5])
smp_id_list = df_["fn"].unique().tolist()

In [17]:
smp_id_list = [i for i in smp_id_list if not i.startswith("Ses01")]

In [12]:
# from tqdm import tqdm 

# fps=30
# for vid in tqdm(smp_id_list):
#     save_video_array(vid,
#                  "E:/datasets/preprocessed/face_raw/{}".format(vid.split("_")[0][:-1]),
#                  fps)

100%|██████████████████████████████████████████████| 123/123 [3:15:53<00:00, 95.55s/it]


In [20]:
save_video_array(smp_id_list[0],
                 "E:/datasets/preprocessed/test/{}".format(smp_id_list[0].split("_")[0][:-1]),
                 fps)

[[198 139 134]
 [198 141 135]
 [196 142 135]
 [197 141 135]
 [198 141 136]
 [199 141 138]
 [200 141 138]
 [200 141 138]
 [201 142 138]
 [204 145 140]
 [203 146 140]
 [204 146 140]
 [205 147 141]
 [205 148 142]
 [204 147 144]
 [205 147 144]
 [205 147 145]
 [206 149 146]
 [205 150 146]
 [205 153 148]
 [205 153 148]
 [203 154 145]
 [203 154 143]
 [206 154 146]
 [206 154 151]
 [206 156 150]
 [206 156 150]
 [207 158 152]
 [206 158 152]
 [205 159 153]
 [205 160 153]
 [205 160 153]
 [205 160 154]
 [205 161 154]
 [207 162 156]
 [206 165 158]
 [205 166 159]
 [205 167 161]
 [206 168 162]
 [207 168 164]
 [207 168 165]
 [207 168 165]
 [209 169 165]
 [208 170 163]
 [208 170 164]
 [208 170 162]
 [208 170 162]
 [208 171 163]
 [207 172 163]
 [204 173 162]
 [204 173 163]
 [204 173 161]
 [202 174 161]
 [202 176 162]
 [202 175 165]
 [203 171 166]
 [203 171 166]
 [203 171 166]
 [203 171 166]
 [203 171 166]
 [203 171 166]
 [202 172 166]
 [201 172 166]
 [201 172 166]
 [199 173 165]
 [197 172 163]
 [196 172 

[[196 139 136]
 [196 140 136]
 [195 140 136]
 [196 141 137]
 [196 142 139]
 [195 144 141]
 [195 143 140]
 [197 144 140]
 [197 144 140]
 [198 146 142]
 [198 146 140]
 [199 145 138]
 [198 146 138]
 [198 148 139]
 [198 149 140]
 [199 150 142]
 [201 150 143]
 [202 151 141]
 [202 151 143]
 [202 150 143]
 [203 151 145]
 [204 152 148]
 [204 152 148]
 [204 152 147]
 [204 152 147]
 [203 153 147]
 [202 154 148]
 [204 156 149]
 [203 158 150]
 [203 158 150]
 [203 158 151]
 [204 159 153]
 [205 160 155]
 [205 160 155]
 [205 164 157]
 [205 166 159]
 [205 166 158]
 [205 167 159]
 [206 167 160]
 [206 168 161]
 [207 169 162]
 [207 169 161]
 [207 169 160]
 [208 170 160]
 [206 170 161]
 [206 170 161]
 [206 170 160]
 [206 170 159]
 [206 171 161]
 [204 171 162]
 [204 171 162]
 [204 171 162]
 [204 171 162]
 [204 171 162]
 [204 171 162]
 [201 172 165]
 [201 172 166]
 [201 172 166]
 [201 172 166]
 [201 172 165]
 [201 173 164]
 [200 172 163]
 [200 172 163]
 [200 172 163]
 [198 169 162]
 [198 170 163]
 [197 170 

KeyboardInterrupt: 